In [6]:
subjects = ('201 205 207 208 209 212 213 215 216 217 218 219 222 224 225 227 228 229 230 231 232 234').split() #
#subjects = ('201 202 203 204 205 206 207 208 209 212 213 214 215 216 217 218 219 221 223 224 225 226 228 229 230 231 232 233 234').split() #

# Mapping of subjects to groups
subject_to_group = {
    201: "trained",
    202: "control",
    203: "trained",
    204: "control",
    205: "control",
    206: "control",
    207: "trained",
    208: "control",
    209: "control",
    212: "trained",
    213: "trained",
    214: "trained",
    215: "control",
    216: "trained",
    217: "control",
    218: "control",
    219: "trained",
    221: "trained",
    223: "trained",
    224: "control",
    225: "trained",
    226: "control",
    228: "trained",
    229: "control",
    230: "trained",
    231: "trained",
    232: "control",
    233: "trained",
    234: "control",
}

In [9]:
# Locate files

import os
import glob
import mne
import numpy as np
from scipy.stats import ttest_rel
import pandas as pd
import os.path as op

# Create necessary directories
raw_path = '../../data'
proc_path = '../../processed'
results_path = '../../results'
subjects_dir = '../../subjects'
os.makedirs(proc_path, exist_ok=True)
os.makedirs(results_path, exist_ok=True)
os.makedirs(subjects_dir, exist_ok=True)

# Define file paths and parameters
days = [1, 3]  # List of day IDs
runs = [1, 2]  # List of run IDs

# Function to read fNIRS data
def read_fnirs_data(file_path):
    raw = mne.io.read_raw_nirx(file_path, preload=True)
    return raw

# Function to extract baseline data (first and last minute)
def extract_baseline_data(raw):
    sfreq = int(raw.info['sfreq'])
    baseline_start = raw[:, :sfreq*60]  # First minute
    baseline_end = raw[:, -sfreq*60:]  # Last minute
    return baseline_start[0], baseline_end[0]

# Function to compute significant change per subject per day
def compute_significant_change(subjects, days, runs, l_freq=0.01, h_freq=0.2):
    results = []
    for subject in subjects:
        for day in days:
            for run in runs:
                group = subject_to_group.get(int(subject), "unknown")
                root1 = f'Day{day}'
                root2 = f'{subject}_{day}'
                root3 = f'*-*-*_{run:03d}'
                fname_base = op.join(raw_path, root1, root2, root3)
                fname = glob.glob(fname_base)
                if not fname:
                    continue
                base = f'{subject}_{day}_{run:03d}'
                raw_intensity = read_fnirs_data(fname[0])
                baseline_start, baseline_end = extract_baseline_data(raw_intensity)
                
                # Compute the mean and standard deviation of the data
                mean_start = np.mean(baseline_start, axis=1)
                mean_end = np.mean(baseline_end, axis=1)
                
                # Perform a paired t-test
                t_stat, p_val = ttest_rel(mean_start, mean_end)
                
                results.append({
                    'subject': subject,
                    'day': day,
                    'run': run,
                    'group': group,
                    'mean_start': mean_start,
                    'mean_end': mean_end,
                    't_stat': t_stat,
                    'p_val': p_val
                })
    
    return results

# Compute significant change
results = compute_significant_change(subjects, days, runs)

# Print results
for result in results:
    print(f"Subject {result['subject']}, Day {result['day']}, Run {result['run']} - T-statistic: {result['t_stat']}, P-value: {result['p_val']}")

# Save the results into a CSV file
df = pd.DataFrame(results)
df.to_csv(op.join(results_path, 'fnirs_baseline_changes.csv'), index=False)


Loading ../../data/Day1/201_1/2023-09-21_001


Reading 0 ... 3994  =      0.000 ...   830.752 secs...
Loading ../../data/Day1/201_1/2023-09-21_002
Reading 0 ... 3838  =      0.000 ...   798.304 secs...
Loading ../../data/Day3/201_3/2023-09-23_001
Reading 0 ... 3945  =      0.000 ...   820.560 secs...
Loading ../../data/Day3/201_3/2023-09-23_002
Reading 0 ... 3945  =      0.000 ...   820.560 secs...
Loading ../../data/Day1/205_1/2023-10-13_001
Reading 0 ... 4171  =      0.000 ...   867.568 secs...
Loading ../../data/Day1/205_1/2023-10-13_002
Reading 0 ... 4933  =      0.000 ...  1026.064 secs...
Loading ../../data/Day3/205_3/2023-10-15_001
Reading 0 ... 3864  =      0.000 ...   803.712 secs...
Loading ../../data/Day3/205_3/2023-10-15_002
Reading 0 ... 3943  =      0.000 ...   820.144 secs...
Loading ../../data/Day1/207_1/2023-10-17_001
Reading 0 ... 3929  =      0.000 ...   817.232 secs...
Loading ../../data/Day1/207_1/2023-10-17_002
Reading 0 ... 3958  =      0.000 ...   823.264 secs...
Loading ../../data/Day3/207_3/2023-10-19_001
